In [1]:
from IPython.display import HTML

html_code = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Wave Equation Simulation</title>
    <!-- Tailwind CSS for styling the user interface -->
    <script src="https://cdn.tailwindcss.com"></script>
    <!-- Google Fonts for a cleaner typography -->
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;500;700&display=swap" rel="stylesheet">
    <style>
        /* Custom CSS for the body font and a blurred background effect on the info box */
        body {
            font-family: 'Inter', sans-serif;
        }
        .info-box {
            background-color: rgba(255, 255, 255, 0.9);
            backdrop-filter: blur(5px);
        }
    </style>
</head>
<body class="bg-gray-900 text-gray-100 flex flex-col items-center justify-center min-h-screen p-4">

    <!-- Main container for the simulation interface -->
    <div class="w-full max-w-4xl bg-gray-800 rounded-xl shadow-2xl p-6 md:p-8">
        <h1 class="text-2xl md:text-3xl font-bold text-center text-cyan-400 mb-4">1D Wave Equation Simulation</h1>
        <p class="text-center text-gray-400 mb-6">Using Finite Difference Method (Central Difference in Time & Space)</p>

        <!-- Container for the canvas and overlays -->
        <div class="relative rounded-lg overflow-hidden bg-black shadow-inner">
            <!-- The canvas where the wave animation is drawn -->
            <canvas id="waveCanvas" class="w-full"></canvas>
            <!-- Overlay box to display the current simulation time -->
            <div id="infoDisplay" class="info-box absolute top-2 left-2 p-3 rounded-lg text-gray-900 text-sm md:text-base font-mono">
                <p>Time: <span id="timeValue">0.000</span> s</p>
            </div>
            <!-- Visual indicators for wave speed regions -->
            <div class="absolute bottom-0 left-0 w-full h-1 bg-gradient-to-r from-blue-500 via-blue-500 to-purple-500" style="width: 75%;"></div>
            <div class="absolute bottom-0 right-0 w-1/4 h-1 bg-purple-500"></div>
            <!-- Text labels for key points on the domain -->
            <div class="absolute bottom-2 left-1/2 -translate-x-1/2 text-xs text-white">x=0.5 (Excitation)</div>
            <div class="absolute bottom-2 left-3/4 -translate-x-1/2 text-xs text-white">x=0.75 (Speed Change)</div>
        </div>

        <!-- Container for control buttons -->
        <div class="mt-6 text-center">
            <button id="startStopBtn" class="bg-cyan-500 hover:bg-cyan-600 text-white font-bold py-2 px-6 rounded-lg transition-all duration-300 shadow-lg">
                Start Simulation
            </button>
            <button id="resetBtn" class="bg-gray-600 hover:bg-gray-700 text-white font-bold py-2 px-6 rounded-lg transition-all duration-300 ml-4 shadow-lg">
                Reset
            </button>
        </div>
    </div>

    <script>
        // Wrap the entire script in a function to avoid polluting the global scope
        // and to ensure it runs after the elements are in the DOM.
        function runWaveSimulation() {
            // --- Get DOM Elements ---
            const canvas = document.getElementById('waveCanvas');
            // A check to ensure the canvas element was found before proceeding.
            if (!canvas) {
                console.error("Canvas element not found!");
                return;
            }
            const ctx = canvas.getContext('2d');
            const timeValueEl = document.getElementById('timeValue');
            const startStopBtn = document.getElementById('startStopBtn');
            const resetBtn = document.getElementById('resetBtn');

            // --- Simulation Parameters ---
            const L = 1.0;          // Length of the domain [m]
            const T_MAX = 3.0;      // Total simulation time [s]
            const NX = 201;         // Number of spatial grid points (higher means better resolution)
            const DX = L / (NX - 1); // Spatial step size
            const MAX_SPEED = 4.0;  // Maximum wave speed in the domain [m/s] (used for stability)

            // --- Stability (Courant-Friedrichs-Lewy or CFL Condition) ---
            // For the FDTD scheme to be stable, the numerical speed of propagation must be
            // at least as fast as the physical speed. This means: c * dt / dx <= 1.
            // To ensure this, we calculate dt based on the MAXIMUM speed in the domain.
            // We multiply by 0.95 to stay safely below the stability limit.
            const DT = (DX / MAX_SPEED) * 0.95; // Time step size [s]
            const NT = Math.floor(T_MAX / DT);  // Total number of time steps

            // --- Excitation Pulse Parameters ---
            const PULSE_START_TIME = 0.1;   // When the pulse begins [s]
            const PULSE_DURATION = 0.5;     // How long the pulse lasts [s]
            const PULSE_END_TIME = PULSE_START_TIME + PULSE_DURATION;
            const EXCITATION_POS_X = 0.5;   // Position of the excitation [m]
            const EXCITATION_INDEX = Math.round(EXCITATION_POS_X / DX); // Grid index for excitation

            // --- Wave Function Arrays ---
            // We need three arrays to represent the wave function at different time steps:
            // u_prev: u at time (t - dt)
            // u_curr: u at time (t)
            // u_next: u at time (t + dt), which we calculate in each step
            let u_prev, u_curr, u_next;

            // --- Wave Speed Array ---
            // This array stores the speed of the wave at each grid point.
            const c = new Float32Array(NX);
            // Pre-calculating this term saves computation time inside the main loop.
            const c_squared_dt_squared_over_dx_squared = new Float32Array(NX);

            // --- Animation State Variables ---
            let animationFrameId;   // ID for the requestAnimationFrame, to cancel it
            let isRunning = false;  // Flag to check if the simulation is running
            let timeStep = 0;       // Current time step index

            /**
             * Initializes or resets the simulation to its starting state.
             */
            function initialize() {
                // Stop any animation that might be running
                if (animationFrameId) {
                    cancelAnimationFrame(animationFrameId);
                }

                // Reset animation state
                isRunning = false;
                timeStep = 0;
                animationFrameId = null;

                // Initialize wave function arrays with all zeros
                u_prev = new Float32Array(NX).fill(0);
                u_curr = new Float32Array(NX).fill(0);
                u_next = new Float32Array(NX).fill(0);

                // Define the variable wave speed and pre-calculate the update coefficient for each point
                for (let i = 0; i < NX; i++) {
                    const x = i * DX; // current position
                    if (x <= 0.75) {
                        c[i] = 1.0; // Slower speed region
                    } else {
                        c[i] = 4.0; // Faster speed region
                    }
                    // This is the core coefficient for the finite difference scheme
                    c_squared_dt_squared_over_dx_squared[i] = Math.pow(c[i] * DT / DX, 2);
                }

                // Set canvas dimensions based on its container's size
                canvas.width = canvas.clientWidth;
                canvas.height = canvas.clientWidth / 2; // Maintain a 2:1 aspect ratio

                // Perform an initial draw and reset UI elements
                draw();
                timeValueEl.textContent = '0.000';
                startStopBtn.textContent = 'Start Simulation';
                startStopBtn.classList.remove('bg-red-500', 'hover:bg-red-600');
                startStopBtn.classList.add('bg-cyan-500', 'hover:bg-cyan-600');
            }

            /**
             * The main simulation loop, called for each frame of the animation.
             */
            function simulationLoop() {
                // Stop the loop if paused or if the simulation has finished
                if (!isRunning || timeStep >= NT) {
                    isRunning = false;
                    startStopBtn.textContent = 'Start Simulation';
                    startStopBtn.classList.remove('bg-red-500', 'hover:bg-red-600');
                    startStopBtn.classList.add('bg-cyan-500', 'hover:bg-cyan-600');
                    return;
                }

                // --- Core Finite Difference Calculation ---
                // This loop implements the discretized wave equation:
                // u_next[i] = 2*u_curr[i] - u_prev[i] + (c*dt/dx)^2 * (u_curr[i+1] - 2*u_curr[i] + u_curr[i-1])
                // It iterates over the interior points of the grid.
                for (let i = 1; i < NX - 1; i++) {
                    const laplacian_u = u_curr[i + 1] - 2 * u_curr[i] + u_curr[i - 1];
                    u_next[i] = 2 * u_curr[i] - u_prev[i] + c_squared_dt_squared_over_dx_squared[i] * laplacian_u;
                }

                // --- Apply Excitation Pulse ---
                const currentTime = timeStep * DT;
                if (currentTime >= PULSE_START_TIME && currentTime < PULSE_END_TIME) {
                    // A sine half-wave is used for a smooth pulse to avoid high-frequency noise
                    const pulse_time = currentTime - PULSE_START_TIME;
                    u_next[EXCITATION_INDEX] = Math.sin( (pulse_time / PULSE_DURATION) * Math.PI );
                }

                // --- Boundary Conditions (Homogeneous Dirichlet) ---
                // u(0, t) = 0 and u(L, t) = 0.
                // Since the arrays are initialized to 0 and the calculation loop skips the
                // first (i=0) and last (i=NX-1) points, these boundaries remain fixed at 0.

                // --- Update arrays for the next time step ---
                // The current state becomes the previous state, and the newly calculated state becomes current.
                u_prev = u_curr;
                u_curr = u_next;
                u_next = new Float32Array(NX).fill(0); // Clear the 'next' array for the next iteration

                // --- Draw the current state and update the time display ---
                draw();
                timeValueEl.textContent = currentTime.toFixed(3);

                // --- Advance to the next step ---
                timeStep++;
                animationFrameId = requestAnimationFrame(simulationLoop);
            }

            /**
             * Draws the current state of the wave function onto the canvas.
             */
            function draw() {
                // Clear the canvas with a dark background color
                ctx.fillStyle = '#111827';
                ctx.fillRect(0, 0, canvas.width, canvas.height);

                // --- Draw the wave itself ---
                ctx.beginPath();
                ctx.strokeStyle = '#22d3ee'; // A bright cyan color
                ctx.lineWidth = 3;
                ctx.lineJoin = 'round'; // For smooth corners

                const midY = canvas.height / 2; // The vertical center of the canvas (y=0 line)
                const ampScale = canvas.height / 4; // Scale the amplitude for better visualization

                for (let i = 0; i < NX; i++) {
                    const x_pos = (i / (NX - 1)) * canvas.width;
                    const y_pos = midY - u_curr[i] * ampScale; // Invert y-axis for standard plot
                    if (i === 0) {
                        ctx.moveTo(x_pos, y_pos);
                    } else {
                        ctx.lineTo(x_pos, y_pos);
                    }
                }
                ctx.stroke();

                // --- Draw the horizontal zero-line for reference ---
                ctx.beginPath();
                ctx.strokeStyle = 'rgba(255, 255, 255, 0.2)';
                ctx.lineWidth = 1;
                ctx.moveTo(0, midY);
                ctx.lineTo(canvas.width, midY);
                ctx.stroke();

                // --- Draw a dashed vertical line to indicate the change in wave speed ---
                ctx.beginPath();
                ctx.strokeStyle = 'rgba(168, 85, 247, 0.5)'; // A semi-transparent purple
                ctx.setLineDash([5, 5]); // Create a dashed line effect
                const interfaceX = (0.75 / L) * canvas.width;
                ctx.moveTo(interfaceX, 0);
                ctx.lineTo(interfaceX, canvas.height);
                ctx.stroke();
                ctx.setLineDash([]); // Reset to solid lines for other drawings
            }

            // --- Event Listeners for Buttons ---
            startStopBtn.addEventListener('click', () => {
                if (isRunning) {
                    // If running, pause the simulation
                    isRunning = false;
                    cancelAnimationFrame(animationFrameId);
                    startStopBtn.textContent = 'Resume';
                    startStopBtn.classList.remove('bg-red-500', 'hover:bg-red-600');
                    startStopBtn.classList.add('bg-cyan-500', 'hover:bg-cyan-600');
                } else {
                    // If paused or stopped, start/resume the simulation
                    if (timeStep >= NT) { // If simulation finished, reset before starting
                        initialize();
                    }
                    isRunning = true;
                    requestAnimationFrame(simulationLoop);
                    startStopBtn.textContent = 'Pause';
                    startStopBtn.classList.remove('bg-cyan-500', 'hover:bg-cyan-600');
                    startStopBtn.classList.add('bg-red-500', 'hover:bg-red-600');
                }
            });

            resetBtn.addEventListener('click', initialize);

            // No resize handler is needed in Colab as the output iframe size is generally fixed.

            // --- Initial Call ---
            // Set up the simulation when the script first loads.
            initialize();
        }

        // Run the main simulation function. This is the entry point of the script.
        runWaveSimulation();
    </script>
</body>
</html>
"""


# This will render the HTML and execute the JavaScript.
HTML(html_code)